In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.tools import tool

In [3]:
import requests
from pydantic.v1 import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [5]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            wikipedia.exceptions.PageError,
            wikipedia.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [6]:
tools = [get_current_temperature, search_wikipedia]

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [8]:
functions = [format_tool_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0).bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [9]:
result = chain.invoke({"input": "what is the weather is sf?"})

In [10]:
result.tool

'get_current_temperature'

In [11]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [12]:
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [13]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [14]:
result1 = chain.invoke({
    "input": "what is the weather is sf?",
    "agent_scratchpad": []
})

In [15]:
result1.tool

'get_current_temperature'

In [16]:
result1

AgentActionMessageLog(tool='get_current_temperature', tool_input={'latitude': 37.7749, 'longitude': -122.4194}, log="\nInvoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}', 'name': 'get_current_temperature'}})])

In [17]:
observation = get_current_temperature(result1.tool_input)

In [18]:
observation

'The current temperature is 11.2°C'

In [19]:
type(result1)

langchain_core.agents.AgentActionMessageLog

In [20]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [21]:
result1.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}', 'name': 'get_current_temperature'}})]

In [22]:
format_to_openai_functions([(result1, observation), ])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}', 'name': 'get_current_temperature'}}),
 FunctionMessage(content='The current temperature is 11.2°C', name='get_current_temperature')]

In [23]:
result2 = chain.invoke({
    "input": "what is the weather is sf?", 
    "agent_scratchpad": format_to_openai_functions([(result1, observation)])
})

In [24]:
result2

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 11.2°C.'}, log='The current temperature in San Francisco is 11.2°C.')

In [25]:
from langchain.schema.agent import AgentFinish
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input, 
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [26]:
from langchain.schema.runnable import RunnablePassthrough
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [27]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input, 
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [28]:
run_agent("Based on the current weather in SF, what clothes should I wear?")

AgentFinish(return_values={'output': 'Based on the current temperature in San Francisco (10.3°C), I would recommend wearing a light jacket or sweater to stay warm. You can also consider layering your clothing to adjust to any changes in temperature throughout the day.'}, log='Based on the current temperature in San Francisco (10.3°C), I would recommend wearing a light jacket or sweater to stay warm. You can also consider layering your clothing to adjust to any changes in temperature throughout the day.')

In [29]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [30]:
agent_executor.invoke({"input": "Based on the current weather in Denver, what should I wear?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 39.7392, 'longitude': -104.9903}`


The current temperature is 1.4°CBased on the current weather in Denver, I would recommend wearing a warm jacket, hat, and gloves. It's quite chilly with a temperature of 1.4°C. Stay warm!

> Finished chain.


{'input': 'Based on the current weather in Denver, what should I wear?',
 'output': "Based on the current weather in Denver, I would recommend wearing a warm jacket, hat, and gloves. It's quite chilly with a temperature of 1.4°C. Stay warm!"}

In [31]:
## Add memory
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [32]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [40]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

In [41]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [42]:
agent_executor.invoke({"input": "Hey! My name is Karan!"})



> Entering new AgentExecutor chain...
Hello Karan! How can I assist you today?

> Finished chain.


{'input': 'Hey! My name is Karan!',
 'chat_history': [HumanMessage(content='Hey! My name is Karan!'),
  AIMessage(content='Hello Karan! How can I assist you today?')],
 'output': 'Hello Karan! How can I assist you today?'}

In [43]:
agent_executor.invoke({"input": "What is my name?"})



> Entering new AgentExecutor chain...
Your name is Karan.

> Finished chain.


{'input': 'What is my name?',
 'chat_history': [HumanMessage(content='Hey! My name is Karan!'),
  AIMessage(content='Hello Karan! How can I assist you today?'),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Karan.')],
 'output': 'Your name is Karan.'}